In [74]:
%load_ext autoreload
%autoreload 2

import pdfplumber
import re

from utils import create_new_sheet, write_content, format_sheet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
############ Update the following parameters ##################
FILEPATH = "../data/June statement.pdf"
NEW_SHEET_NAME = "June 2024"

In [76]:
SPREADSHEET_ID = "1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY"

In [77]:
pdf_lines = []
with pdfplumber.open(FILEPATH) as pdf:
    for page_id in range(len(pdf.pages)):
        page = pdf.pages[page_id]
        pdf_lines.extend(page.extract_text_lines())
pdf_lines = [line['text'] for line in pdf_lines]

In [78]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# lr_model = Pipeline([('vect', CountVectorizer()),
#                ('tfidf', TfidfTransformer()),
#                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
#               ])

lr_model = pickle.load(open('pickle/lr_model.pkl', 'rb'))

In [79]:
id_to_label = {0: 'Regular', 1: 'Trip', 2: 'Hobby'}

In [80]:
regex_pattern = "^(\d\d/\d\d.*) (-*[\d\.,]+)$"
accumulated_content = []
for line in pdf_lines:
    if "AUTOMATIC PAYMENT" in line:
        continue
    re_search = re.search(regex_pattern, line)
    if re_search is not None:
        transaction_name = re_search.group(1)
        expense_type_id = lr_model.predict([transaction_name])[0]
        expense_type = id_to_label[expense_type_id] if expense_type_id in id_to_label else 'Regular'
        cost = float(re_search.group(2).replace(',', ''))
        accumulated_content.append([transaction_name, cost, expense_type])

In [81]:
# Sort the content according to the transaction date
accumulated_content = sorted(accumulated_content, key=lambda x: x[0].split(' ')[0])

for i in range(len(accumulated_content)):
    row_id = i + 2
    regular_cost = f'=IF(C{row_id}="Regular", B{row_id}, 0)'
    trip_cost = f'=IF(C{row_id}="Trip", B{row_id}, 0)'
    hobby_cost = f'=IF(C{row_id}="Hobby", B{row_id}, 0)'
    accumulated_content[i].append(regular_cost)
    accumulated_content[i].append(trip_cost)
    accumulated_content[i].append(hobby_cost)

accumulated_content = [['Transaction', 'Cost', 'Type', 'Regular expenses', 'Trip expenses', 'Hobby expenses']] + accumulated_content + [['', f'=SUM(B2:B{row_id})', '', f'=SUM(D2:D{row_id})', f'=SUM(E2:E{row_id})', f'=SUM(F2:F{row_id})', f'=SUM(D{row_id+1}:F{row_id+1})']]

In [82]:
# Sort the content according to the transaction date
# accumulated_content = [accumulated_content[0]] + sorted(accumulated_content[1:-1], key=lambda x: x[0].split(' ')[0]) + [accumulated_content[-1]]

In [83]:
# Export to the spreadsheet.
create_new_sheet_response = create_new_sheet(SPREADSHEET_ID, NEW_SHEET_NAME)
sheet_id = create_new_sheet_response['replies'][0]['addSheet']['properties']['sheetId']
write_response = write_content(SPREADSHEET_ID, NEW_SHEET_NAME, accumulated_content)
format_sheet_response = format_sheet(SPREADSHEET_ID, sheet_id)

Created a new sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{'addSheet': {'properties': {'sheetId': 1203295335, 'title': 'June 2024', 'index': 12, 'sheetType': 'GRID', 'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}}]}
Exported content to 433 cells.
Format sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{}, {}, {}]}
